In [ ]:
from itertools import product

def discount_value(amount, t, r):
    factor = (1 + r) ** t
    if isinstance(amount, tuple) and amount[0] == 'interval':
        return ('interval', amount[1]/factor, amount[2]/factor)
    else:
        return amount/factor if not isinstance(amount, tuple) else (amount[0]/factor, amount[1]/factor)

def add_values(v1, v2):
    # Add two values that can be discrete or ('interval', a, b)
    if isinstance(v1, tuple) and v1[0] == 'interval' and isinstance(v2, tuple) and v2[0] == 'interval':
        return ('interval', v1[1]+v2[1], v1[2]+v2[2])
    elif isinstance(v1, tuple) and v1[0] == 'interval' and not (isinstance(v2, tuple) and v2[0] == 'interval'):
        # v2 discrete
        return ('interval', v1[1]+v2, v1[2]+v2)
    elif not (isinstance(v1, tuple) and v1[0] == 'interval') and isinstance(v2, tuple) and v2[0] == 'interval':
        # v1 discrete, v2 interval
        return ('interval', v1+v2[1], v1+v2[2])
    else:
        # both discrete
        return v1+v2

def min_value_of_set(h_set):
    mins = []
    for val in h_set:
        if isinstance(val, tuple) and val[0] == 'interval':
            mins.append(val[1])
        else:
            mins.append(val)
    return min(mins) if mins else None

def max_value_of_set(h_set):
    maxs = []
    for val in h_set:
        if isinstance(val, tuple) and val[0] == 'interval':
            maxs.append(val[2])
        else:
            maxs.append(val)
    return max(maxs) if maxs else None

def form_interval_set(values):
    """
    Given a list of values (discrete or intervals), we return a simplified union.
    We will just keep them as a list. If desired, one could try merging overlapping intervals.
    """
    # Remove duplicates
    unique_values = []
    for v in values:
        if v not in unique_values:
            unique_values.append(v)
    return unique_values

def scale_set(h_set, factor):
    """
    Multiplies every element of h_set by 1/factor (to get C from P or vice versa).
    """
    scaled = []
    for val in h_set:
        if isinstance(val, tuple) and val[0] == 'interval':
            scaled.append(('interval', val[1]/factor, val[2]/factor))
        else:
            scaled.append(val/factor)
    return scaled

def compute_p_next(C_set, n, r):
    """
    Compute P_{K+1} given C_K.
    We consider all combinations of elements from C_set over n periods and compute their PV.
    """
    # C_set is a hesitant set (list) of discrete and/or intervals
    # For simplicity, we form n identical sets (since it's a constant amortization term scenario)
    # Actually, here we consider that each period can choose any c from C_set (like maximum uncertainty).

    # However, if the method implies constant installments (the same c each period),
    # we would just consider each c in C_set and compute PV.
    # The definition suggests we consider all combos (c_1,...,c_n), each from C_set.

    all_combinations = product(C_set, repeat=n)
    result_values = []
    for combo in all_combinations:
        # combo is (c_1, c_2, ..., c_n)
        # compute PV = sum c_i/(1+r)^i
        pv = combo[0]
        # Discount and sum them properly:
        # The first element should also be discounted at t=1
        pv = discount_value(pv, 1, r)
        for i, c in enumerate(combo[1:], start=2):
            discounted = discount_value(c, i, r)
            pv = add_values(pv, discounted)
        result_values.append(pv)

    return form_interval_set(result_values)

def print_hesitant_set(h_set):
    # Simple printer
    # Sort by min value of interval or value if discrete
    def sort_key(x):
        if isinstance(x, tuple) and x[0] == 'interval':
            return x[1]
        else:
            return x
    sorted_vals = sorted(h_set, key=sort_key)
    out = []
    for val in sorted_vals:
        if isinstance(val, tuple) and val[0] == 'interval':
            out.append(f"[{val[1]:.2f}, {val[2]:.2f}]")
        else:
            out.append(f"{val:.2f}")
    return " ∪ ".join(out)

if __name__ == "__main__":
    # Example as in the text:
    # P_1 = {1000} U [1100,1200]
    # n = 2, r = 0.05
    # S = (1 - (1.05)^(-2))/0.05 = 1.85941 (approx)

    P_1 = [1000, ('interval', 1100, 1200)]
    n = 2
    r = 0.05
    K_max = 5  # number of iterations
    # Compute S
    S = (1 - (1+r)**(-n))/r

    P_current = P_1
    for K in range(1, K_max+1):
        # Compute C_K
        C_K = scale_set(P_current, S)  # C = P / S for each element in P
        # Compute P_{K+1}
        P_next = compute_p_next(C_K, n, r)

        print(f"Iteration K={K}:")
        print(f"P_{K} = {print_hesitant_set(P_current)}")
        print(f"C_{K} = {print_hesitant_set(C_K)}")
        print(f"P_{K+1} = {print_hesitant_set(P_next)}")
        print("----")

        P_current = P_next


Iteration K=1:
P_1 = 1000.00 ∪ [1100.00, 1200.00]
C_1 = 537.80 ∪ [591.59, 645.37]
P_2 = 1000.00 ∪ [1048.78, 1097.56] ∪ [1051.22, 1102.44] ∪ [1100.00, 1200.00]
----
Iteration K=2:
P_2 = 1000.00 ∪ [1048.78, 1097.56] ∪ [1051.22, 1102.44] ∪ [1100.00, 1200.00]
C_2 = 537.80 ∪ [564.04, 590.27] ∪ [565.35, 592.90] ∪ [591.59, 645.37]
P_3 = 1000.00 ∪ [1023.80, 1047.59] ∪ [1024.99, 1049.97] ∪ [1026.23, 1052.47] ∪ [1048.78, 1097.56] ∪ [1048.78, 1097.56] ∪ [1049.97, 1099.94] ∪ [1050.03, 1100.06] ∪ [1051.22, 1102.44] ∪ [1073.77, 1147.53] ∪ [1075.01, 1150.03] ∪ [1075.01, 1150.03] ∪ [1076.20, 1152.41] ∪ [1100.00, 1200.00]
----
Iteration K=3:
P_3 = 1000.00 ∪ [1023.80, 1047.59] ∪ [1024.99, 1049.97] ∪ [1026.23, 1052.47] ∪ [1048.78, 1097.56] ∪ [1048.78, 1097.56] ∪ [1049.97, 1099.94] ∪ [1050.03, 1100.06] ∪ [1051.22, 1102.44] ∪ [1073.77, 1147.53] ∪ [1075.01, 1150.03] ∪ [1075.01, 1150.03] ∪ [1076.20, 1152.41] ∪ [1100.00, 1200.00]
C_3 = 537.80 ∪ [550.60, 563.40] ∪ [551.24, 564.68] ∪ [551.91, 566.02] ∪ [564.04,

KeyboardInterrupt: 